In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from ftplib import FTP_TLS
import tkinter as tk
from tkinter import messagebox

delta=dt.timedelta(days=1)

In [ ]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,tipo):

     if tipo=='WeekDay':
          return row['Fecha'].weekday()
     
     elif tipo=='WeekMonth':
          return (row['Fecha'].day - 1) // 7 + 1
     
     elif tipo=='DayType':
          if row['Fecha'] in co_holidays:
               return 3
          elif row['Fecha'].weekday()==5:
               return 2
          elif row['Fecha'].weekday()==6:
               return 3
          else:
               return 1

In [ ]:
from pathlib import Path
import pandas as pd

def cargar_dgs_max(
    carpeta: str | Path,
    anio_ini: int = 2026,
    anio_fin: int = 2036,
) -> pd.DataFrame:
    """
    Lee archivos tipo: 2026_Max.dgs, 2027_Max.dgs, ..., 2036_Max.dgs
    - Elimina (omite) las 3 primeras filas de cada archivo
    - Separa por ';'
    - Devuelve un DataFrame consolidado con columnas: Carga, ourserv, P, Q, anio
    """
    carpeta = Path(carpeta)
    dfs = []

    for anio in range(anio_ini, anio_fin + 1):
        archivo = carpeta / f"{anio}_Max.dgs"

        if not archivo.exists():
            print(f"[WARN] No existe: {archivo}")
            continue

        # Lee el archivo: 4 columnas separadas por ';', omite 3 filas iniciales
        df = pd.read_csv(
            archivo,
            sep=";",
            skiprows=3,
            header=None,
            names=["Carga", "ourserv", "P", "Q"],
            engine="python",
            encoding="utf-8",
        )

        # Limpieza básica
        df = df.dropna(how="all")  # filas totalmente vacías
        df["Carga"] = df["Carga"].astype(str).str.strip()

        # Convierte numéricos (por si hay espacios o strings raros)
        for col in ["ourserv", "P", "Q"]:
            df[col] = pd.to_numeric(df[col], errors="coerce")

        df["anio"] = anio
        dfs.append(df)

    if not dfs:
        return pd.DataFrame(columns=["Carga", "ourserv", "P", "Q", "anio"])

    return pd.concat(dfs, ignore_index=True)


# ===== Ejemplo de uso =====
# carpeta_datos = r"C:\ruta\a\los\archivos"
# df_all = cargar_dgs_max(carpeta_datos, 2026, 2036)
# print(df_all.head())
# print(df_all.tail())


In [4]:
# ===== Ejemplo de uso =====
carpeta_datos = r"C:\Información XM\Publico\PlaneacionOperacion\MedianoPlazo\BasesDatosPowerFactoryMP\2025\T4\DGS Demanda\Nacional\\"
df_all = cargar_dgs_max(carpeta_datos, 2026, 2036)
print(df_all.head())
print(df_all.tail())

        Carga  ourserv          P         Q  anio
0  ##CrgARc11        0   8.279882  2.242464  2026
1  ##CrgAaz11        0  13.230547  4.059905  2026
2  ##CrgAca11        0  20.696687  8.533896  2026
3  ##CrgAce11        0   5.132916  2.232774  2026
4  ##CrgAch11        0  18.347883  6.552937  2026
           Carga  ourserv          P         Q  anio
5374  ##CrgZip11        0  33.726294  8.578332  2036
5375  ##CrgZrg61        0  24.362904  8.127888  2036
5376  ##CrgZrg62        0  28.519177  8.870875  2036
5377  ##CrgZrg63        0  23.956389  7.896914  2036
5378  ##CrgZul11        0   6.910693  2.593178  2036


In [6]:
df_all

,Carga,ourserv,P,Q,anio
0,##CrgARc11,0,8.279882,2.242464,2026
1,##CrgAaz11,0,13.230547,4.059905,2026
2,##CrgAca11,0,20.696687,8.533896,2026
3,##CrgAce11,0,5.132916,2.232774,2026
4,##CrgAch11,0,18.347883,6.552937,2026
...,...,...,...,...,...
5374,##CrgZip11,0,33.726294,8.578332,2036
5375,##CrgZrg61,0,24.362904,8.127888,2036
5376,##CrgZrg62,0,28.519177,8.870875,2036
5377,##CrgZrg63,0,23.956389,7.896914,2036


In [11]:
import numpy as np
import pandas as pd

TOL = 0.05  # 2%

# Eliminar posibles NaN en P
df = df_all.dropna(subset=["P"]).copy()

# Función que evalúa estabilidad por carga
def carga_estable(grp):
    P_ref = grp["P"].mean()
    if P_ref == 0:
        return False  # evita división por cero
    desvio_rel = (grp["P"] - P_ref).abs() / P_ref
    return desvio_rel.max() <= TOL

# Identificar cargas estables
cargas_estables = (
    df
    .groupby("Carga", as_index=False)
    .filter(carga_estable)
)

# Lista única de cargas que cumplen la condición
lista_cargas_estables = cargas_estables["Carga"].unique()

print(f"Número de cargas estables: {len(lista_cargas_estables)}")
lista_cargas_estables


Número de cargas estables: 6


array(['##CrgMni11', '##CrgOxy31', '##CrgPDm21', '##CrgSnF21',
       '##CrgTub21', '##CrgUre11'], dtype=object)

In [23]:
import numpy as np
import pandas as pd

TOL = 0.2  # 2%
ANIO_BASE = 2026
ANIO_FIN = 2036

df = df_all.dropna(subset=["P"]).copy()

# P en 2026
p_2026 = (
    df[df["anio"] == ANIO_BASE][["Carga", "P"]]
    .rename(columns={"P": "P_2026"})
    .drop_duplicates(subset=["Carga"])
)

# P en 2036
p_2036 = (
    df[df["anio"] == ANIO_FIN][["Carga", "P"]]
    .rename(columns={"P": "P_2036"})
    .drop_duplicates(subset=["Carga"])
)

# Comparación 2026 vs 2036
comp_2026_2036 = p_2026.merge(p_2036, on="Carga", how="inner")

comp_2026_2036["desv_rel"] = (
    (comp_2026_2036["P_2036"] - comp_2026_2036["P_2026"]).abs()
    / comp_2026_2036["P_2026"].replace({0: np.nan})
)

# Cumple tolerancia ±2%
comp_2026_2036["estable_2pct"] = comp_2026_2036["desv_rel"] < TOL

# Cargas que cumplen
cargas_estables_2026_2036 = comp_2026_2036.loc[
    comp_2026_2036["estable_2pct"], "Carga"
].to_list()


cargas_estables_2026_2036



['##CrgCin21',
 '##CrgMni11',
 '##CrgOxy31',
 '##CrgPDm21',
 '##CrgSnF21',
 '##CrgTub21',
 '##CrgUre11']

In [28]:
import pandas as pd

# df_all: DataFrame original
# cargas_estables_2026_2036: lista ya calculada

df = df_all.copy()

# Excluir cargas estables
df_filtrado = df[~df["Carga"].isin(cargas_estables_2026_2036)]

# Demanda total por año (P y Q)
demanda_total_por_anio = (
    df_filtrado
    .groupby("anio", as_index=False)
    .agg(
        P_total=("P", "sum"),
        Q_total=("Q", "sum"),
    )
    .sort_values("anio")
)

# Resultado
demanda_total_por_anio['P_total']=demanda_total_por_anio['P_total'].round(2)
demanda_total_por_anio['Q_total']=demanda_total_por_anio['Q_total'].round(2)

demanda_total_por_anio.to_csv('DemandaAño.csv')


In [29]:
# Cargas especiales = cargas_estables_2026_2036
ANIO = 2026

cargas_especiales_2026 = (
    df_all[
        (df_all["anio"] == ANIO) &
        (df_all["Carga"].isin(cargas_estables_2026_2036))
    ][["Carga", "P", "Q"]]
    .sort_values("Carga")
    .reset_index(drop=True)
)

# Resultado:
# cargas_especiales_2026 -> DataFrame con P y Q de las cargas especiales en 2026
cargas_especiales_2026


,Carga,P,Q
0,##CrgCin21,52.257153,32.214370
1,##CrgMni11,21.434423,7.197898
2,##CrgOxy31,92.125737,33.671293
3,##CrgPDm21,45.000000,21.794495
4,##CrgSnF21,180.000000,45.112252
5,##CrgTub21,13.000000,2.011495
6,##CrgUre11,168.596460,36.918595
